In [1]:
from typing import TypedDict, Annotated
from langgraph.graph import add_messages, StateGraph, END
from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage, HumanMessage
from dotenv import load_dotenv
# In-memory Checkpointer
from langgraph.checkpoint.memory import MemorySaver

In [2]:
load_dotenv()

True

In [3]:
memory=MemorySaver()

In [6]:
llmLlama=ChatGroq(model="llama-3.1-8b-instant")

In [8]:
class BasicChatState(TypedDict):
    messages: Annotated[list, add_messages]

In [9]:
def chatbot(state: BasicChatState):
    return {
        "messages":[llmLlama.invoke(input=state["messages"])]
    }

In [10]:
graph=StateGraph(state_schema=BasicChatState)

In [11]:
graph.add_node(node="chatbot",action=chatbot)

In [12]:
graph.add_edge(start_key="chatbot",end_key=END)

In [13]:
graph.set_entry_point(key="chatbot")

In [ ]:
app=graph.compile(checkpointer=memory)

In [15]:
app.get_graph().print_ascii()

+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
 +---------+   
 | chatbot |   
 +---------+   
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   


In [16]:
config={"configurable":{"thread_id":1}}

In [17]:
response1=app.invoke(input={"messages":HumanMessage(content="Hi I am Harish")},config=config)

In [18]:
response1

{'messages': [HumanMessage(content='Hi I am Harish', additional_kwargs={}, response_metadata={}, id='cc4dcdad-9701-4353-aa54-ee2fe2735719'),
  AIMessage(content="Hello Harish, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 40, 'total_tokens': 67, 'completion_time': 0.036, 'prompt_time': 0.003450804, 'queue_time': 0.29080754600000003, 'total_time': 0.039450804}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_8d1c73bd4c', 'finish_reason': 'stop', 'logprobs': None}, id='run-07035a45-c45a-4695-ab2b-a29aff840336-0', usage_metadata={'input_tokens': 40, 'output_tokens': 27, 'total_tokens': 67})]}

In [19]:
response2=app.invoke(input={"messages":HumanMessage(content="What's my name?")},config=config)

In [20]:
response2

{'messages': [HumanMessage(content='Hi I am Harish', additional_kwargs={}, response_metadata={}, id='cc4dcdad-9701-4353-aa54-ee2fe2735719'),
  AIMessage(content="Hello Harish, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 40, 'total_tokens': 67, 'completion_time': 0.036, 'prompt_time': 0.003450804, 'queue_time': 0.29080754600000003, 'total_time': 0.039450804}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_8d1c73bd4c', 'finish_reason': 'stop', 'logprobs': None}, id='run-07035a45-c45a-4695-ab2b-a29aff840336-0', usage_metadata={'input_tokens': 40, 'output_tokens': 27, 'total_tokens': 67}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='b26b9e32-461a-4c24-8edb-f472c8fdac31'),
  AIMessage(content='Your name is Harish.', additional_kwargs={}, response_metadata={'token_usage': {'completion_t

In [39]:
# get snapshot of the latest checkpoint
snapshot=app.get_state(config=config)

In [21]:
snapshot

NameError: name 'snapshot' is not defined

In [22]:
while True:
    user_input=input("User: ")
    if (user_input in ["exit","end"]):
        break
    else:
        result=app.invoke(input={
            "messages":HumanMessage(content=user_input)
        },config=config)
        print(result['messages'][-1].content)

Since my knowledge cutoff is December 2023, I can provide information up to that date. As of my knowledge cutoff, the President of the United States was Joe Biden. However, please note that the presidency may have changed since my knowledge cutoff.
As of my knowledge cutoff in December 2023, Narendra Modi is the Prime Minister of India. He has been serving as the Prime Minister since 2014, and his term has been extended due to the 2019 general elections, where his party, the Bharatiya Janata Party (BJP), won a majority.
I think there might be some confusion earlier. You initially told me your name was Harish, but now you've mentioned it's actually Ritish.
Your name is Ritish.


In [ ]:
while True:
    user_input=input("User: ")
    if (user_input in ["exit","end"]):
        break
    else:
        result=app.invoke(input={
            "messages":HumanMessage(content=user_input)
        },config=config)
        print(result['messages'][-1].content)

User:  What's my name?


Your name is Harish.
